In [ ]:
"""
Run k-means
"""

from gasp.gt.nop.cls import k_means

k_means(
    '/mnt/d/gpto/caplivro_landscape/carta_clip.tif',
    '/mnt/d/gpto/caplivro_landscape/carta_kmeans.tif', 12
)

In [ ]:
train_ref = '/home/jasp/mrgis/clsimg/lsb_trainf2.tif'

"""
class_proportion = {
    1 : 34.15,
    2 : 12.95,
    3 : 5.40,
    4 : 19.49,
    5 : 5.84,
    6 : 0.53,
    7 : 1.13,
    8 : 20.51
}"""

out_rst = '/home/jasp/mrgis/clsimg/lsb_trainf2_rs_1000k_v2.tif'

sample_dim = 1000000
cls_sample_min = 20000

from glass.geo.gt.sample import proprndcells_to_rst

proprndcells_to_rst(train_ref, class_proportion, out_rst, sample_dim, cls_sample_min=cls_sample_min)

In [ ]:
# Lisboa
var = [
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20181022.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20180321.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20180619.tif',
    #'/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20181022.tif',
]

In [ ]:
# PNSE
var = [
    '/home/jasp/mrgis/clsimg/se_imgcls/b02_20180326.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b02_20180619.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b02_20181022.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b03_20180326.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b03_20180619.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b03_20181022.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b04_20180326.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b04_20180619.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b04_20181022.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b08_20180326.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b08_20180619.tif',
    '/home/jasp/mrgis/clsimg/se_imgcls/b08_20181022.tif'
]

In [ ]:
from glass.geo.gt.nop.cls import train_to_mdl

ref = '/home/jasp/mrgis/clsimg/lsb_trainf2_rs_1000k_v2.tif'

mdlf = '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f2_500k_4x3_05kt.bz2'

refs = [
    '/home/jasp/mrgis/clsimg/lsb_trainf1_rs_1000k_v2.tif',
    '/home/jasp/mrgis/clsimg/lsb_trainf2_rs_1000k_v2.tif'
]

mdlf = [
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f1_1000k_4x3_05kt.bz2',
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f2_1000k_4x3_05kt.bz2'
]

for i in range(len(refs)):
    train_to_mdl(refs[i], var, mdlf[i], ntrees=500)

In [ ]:
mdlfile = [
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f0_1000k_4x3_05kt.bz2',
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f1_1000k_4x3_05kt.bz2',
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f2_1000k_4x3_05kt.bz2'
]
    

outfile = [
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f0_1000k_4x3_05kt_v206.tif',
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f1_1000k_4x3_05kt_v206.tif',
    '/home/jasp/mrgis/clsimg/rf_res_206/lsb_f2_1000k_4x3_05kt_v206.tif'
]

from glass.geo.gt.nop.cls import imgcls_from_mdl

for i in range(len(mdlfile)):
    imgcls_from_mdl(mdlfile[i], var, outfile[i])

In [ ]:
"""
Random forest - Procude model file
"""

train_ref = '/home/jasp/mrgis/clsimg/rst_train_f2.tif'

train_var = [
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b05_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b06_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b07_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b8a_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b11_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b12_20181022.tif',
]

ntrees     = 2000
sample_dim = 500000

outfile = '/home/jasp/mrgis/clsimg/mdl_f2_12b_500k_2k.bz2'

from glass.geo.gt.nop.cls import random_train_to_mdl

random_train_to_mdl(train_ref, train_var, outfile, sample_dim=sample_dim, ntrees=ntrees)